# 4.3 by 7 mm multi-qubit chip

In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
import numpy as np

In [2]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL

from qiskit_metal.qlibrary.couplers.tunable_coupler_01 import TunableCoupler01

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

# from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
# from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

In [3]:
# Create a DesignPlanar (class to handle 2D designs)
design = metal.designs.DesignPlanar(metadata = Dict(design_name='FourQubitDesign'))
design.overwrite_enabled = True # Enables overwriting components

gui = MetalGUI(design)

In [4]:
# specify the path of junction files
a_gds = design.renderers.gds

a_gds.options['path_filename'] = 'C:\Jupyter_notebooks\chip_design_base_env/FakeJunction_01.GDS'
# a_gds.options

## Chip Size

In [5]:
from SQDMetal.Utilities.QUtilities import QUtilities

design.chips.main.size.size_x = '7.5mm'
design.chips.main.size.size_y = '5.0mm'
design.chips.main.size.size_z = '500um'
design.chips.main.size.center_x = '3.55mm'
design.chips.main.size.center_y = '2.2mm'

## EBL 20um by 20um alignment markers

In [6]:
# from SQDMetal.Comps.Markers import MarkerDicingCross, MarkerSquare4, MarkerSquare

# MarkerDicingCross(design, 'mkr_dicing_1', Dict(pos_x='0mm', pos_y='0mm'))
# MarkerDicingCross(design, 'mkr_dicing_2', Dict(pos_x='7.1mm', pos_y='0mm'))
# MarkerDicingCross(design, 'mkr_dicing_3', Dict(pos_x='7.1mm', pos_y='4.4mm'))
# MarkerDicingCross(design, 'mkr_dicing_4', Dict(pos_x='0mm', pos_y='4.4mm'))

# align_layer=5
# MarkerSquare4(design, 'mkr1', options=Dict(pos_x='0.35mm', pos_y='0mm', layer=align_layer))
# MarkerSquare4(design, 'mkr2', options=Dict(pos_x='6.75mm', pos_y='0mm', layer=align_layer))
# MarkerSquare4(design, 'mkr3', options=Dict(pos_x='0.35mm', pos_y='4.4mm', layer=align_layer))
# MarkerSquare4(design, 'mkr4', options=Dict(pos_x='6.75mm', pos_y='4.4mm', layer=align_layer))

# MarkerSquare(design, 'mkr_align_chk', options=Dict(pos_x='0.48mm', pos_y='0.19mm', square_width='40um', square_height='40um', layer=1));
# MarkerSquare(design, 'mkr_align_chk2', options=Dict(pos_x='0.16mm', pos_y='0.19mm', square_width='40um', square_height='40um', layer=2));
# MarkerSquare(design, 'mkr_align_chk3', options=Dict(pos_x='0.16mm', pos_y='-0.13mm', square_width='40um', square_height='40um', layer=3));

## Launcher Pads

Launcher pads need to be placed 1.0mm from the four corners of the chip ie from the center of the dicing cross. for alignment with pcb. For refernce see chip_sizes pdf

In [7]:
from SQDMetal.Utilities.CpwParams import CpwParams
from SQDMetal.Comps.Joints import JointExtend

launcher_width = 300e-6#250e-6
launcher_length = 666e-6
cpw_width = 10e-6

cpw = CpwParams(11.7, 500e-6)
launcher_gap = cpw.get_gap_from_width(launcher_width)
cpw_gap = cpw.get_gap_from_width(cpw_width)

launch_nw = LaunchpadWirebond(design, 'Launch_NW',options = dict(pos_x = '0.66mm', pos_y ='2.2mm', orientation = '0',
                                                                 pad_width = '400um', pad_height = f'{launcher_width*1e6}um', pad_gap = f'{launcher_gap*1e6}um',  taper_height = '250um',
                                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))

launch_ne = LaunchpadWirebond(design, 'Launch_NE',options = dict(pos_x = '6.44mm', pos_y='2.2mm', orientation=180 , 
                                                                 pad_width = '400um', pad_height = f'{launcher_width*1e6}um', pad_gap = f'{launcher_gap*1e6}um', taper_height = '250um',
                                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))

# launch_c = LaunchpadWirebond(design, 'Launch_c',options = dict(pos_x = '6.1mm', pos_y ='3.83mm', orientation = '270', 
#                                                                  pad_width = '400um', pad_height = f'{launcher_width*1e6}um', pad_gap = f'{launcher_gap*1e6}um',  taper_height = '250um',
#                                                                  trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))

# launch_d = LaunchpadWirebond(design, 'Launch_d',options = dict(pos_x = '1.0mm', pos_y='3.83mm', orientation='270' , 
#                                                                  pad_width = '400um', pad_height = f'{launcher_width*1e6}um', pad_gap = f'{launcher_gap*1e6}um', taper_height = '250um',
#                                                                  trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))

# launch_a = LaunchpadWirebond(design, 'Launch_a',options = dict(pos_x = '1.0mm', pos_y ='0.57mm', orientation = '90', 
#                                                                  pad_width = '400um', pad_height = f'{launcher_width*1e6}um', pad_gap = f'{launcher_gap*1e6}um',  taper_height = '250um',
#                                                                  trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))

# launch_b = LaunchpadWirebond(design, 'Launch_b',options = dict(pos_x = '6.1mm', pos_y='0.57mm', orientation='90' , 
#                                                                  pad_width = '400um', pad_height = f'{launcher_width*1e6}um', pad_gap = f'{launcher_gap*1e6}um', taper_height = '250um',
#                                                                  trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))

gui.rebuild()

## Qubits

In [8]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_teeth import TransmonPocketTeeth
from SQDMetal.Utilities.CpwParams import CpwParams

cpw_width = 10e-6
cpw_gap = cpw.get_gap_from_width(cpw_width)


Q4 = TransmonPocketTeeth(design, 'Q4', options = dict(
        pos_x='3.1mm',
        pos_y='1.3mm',
        orientation = '90',

        pad_gap ='38um',                       
        inductor_width    = '20um',                       
        pad_width       = '400um',                      
        pad_height       = '90um',                       
        pocket_width    = '650um',                      
        pocket_height    = '650um',                      
        coupled_pad_height= '65um',                      
        coupled_pad_width = '20um',                       
        coupled_pad_gap   = '50um',

        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        connection_pads = dict(
            readout = dict(loc_W=0, loc_H=1, pad_height = '50um', pad_width= '20um', cpw_extend = '20um', pad_cpw_shift='-0um', pad_gap ='10um',
                            trace_width=f'{cpw_width*1e6}um', cpw_gap=f'{cpw_gap*1e6}um'),
            )))

    # Q5 = TransmonPocketTeeth(design,'Q5', options = dict(
    #     pos_x='3.9mm',
    #     pos_y='1.3mm',
    #     orientation = '270',
        
    #     pad_gap ='30um',                       
    #     inductor_width    = '20um',                       
    #     pad_width       = '400um',                      
    #     pad_height       = '90um',                       
    #     pocket_width    = '650um',                      
    #     pocket_height    = '650um',                      
    #     coupled_pad_height= '30um',                      
    #     coupled_pad_width = '20um',                       
    #     coupled_pad_gap   = '50um',

    #     # gds_cell_name ='FakeJunction_01',
    #     hfss_inductance ='14nH',
    #     connection_pads = dict(
    #         readout = dict(loc_W=0, loc_H=1, pad_height = '50um', pad_width= '20um', cpw_extend = '20um', pad_cpw_shift='-0', pad_gap ='10um',
    #                         trace_width=f'{cpw_width*1e6}um', cpw_gap=f'{cpw_gap*1e6}um'),
    #         )))



    # Q1 = TransmonPocketTeeth(design,'Q1', options = dict(
    #     pos_x='3.1mm',
    #     pos_y='3.0mm',
    #     orientation = '90',
        
    #     pad_gap ='30um',                       
    #     inductor_width    = '20um',                       
    #     pad_width       = '400um',                      
    #     pad_height       = '90um',                       
    #     pocket_width    = '650um',                      
    #     pocket_height    = '650um',                      
    #     coupled_pad_height= '30um',                      
    #     coupled_pad_width = '20um',                       
    #     coupled_pad_gap   = '50um',

    #     # gds_cell_name ='FakeJunction_01',
    #     hfss_inductance ='14nH',
    #     connection_pads = dict(
    #         readout = dict(loc_W=0, loc_H=1, pad_height = '50um', pad_width= '20um', cpw_extend = '20um', pad_cpw_shift='-0', pad_gap ='10um',
    #                         trace_width=f'{cpw_width*1e6}um', cpw_gap=f'{cpw_gap*1e6}um'),
    #         )))
    # # Quasi Particle qubit
    # Q2 = TransmonPocketTeeth(design,'Q2', options = dict(
    #     pos_x='3.9mm',
    #     pos_y='3.05mm',
    #     orientation = '270',
        
    #     pad_gap ='38um',                       
    #     inductor_width    = '20um',                       
    #     pad_width       = '400um',                      
    #     pad_height       = '90um',                       
    #     pocket_width    = '650um',                      
    #     pocket_height    = '650um',                      
    #     coupled_pad_height= '65um',                      
    #     coupled_pad_width = '20um',                       
    #     coupled_pad_gap   = '50um',

    #     # gds_cell_name ='FakeJunction_01',
    #     hfss_inductance ='14nH',
    #     connection_pads = dict(
    #         readout = dict(loc_W=0, loc_H=1, pad_height = '50um', pad_width= '20um', cpw_extend = '20um', pad_cpw_shift='-0um', pad_gap ='10um',
    #                         trace_width=f'{cpw_width*1e6}um', cpw_gap=f'{cpw_gap*1e6}um')
    #         )))

gui.rebuild()

## Feedline to Resonator connections

In [9]:

#We use finger count to set the width of the gap capacitance, -> N*cap_width + (N-1)*cap_gap
#0.7mm gap should be maintained between the ends of read out resonators 
q4_read_T = CoupledLineTee(design,'Q4_Read_T', options=dict(pos_x = '1.0mm', pos_y = '2.2mm',
                                                        orientation = '0', coupling_space='5um',
                                                        coupling_length = '300um', down_length='100um',
                                                        prime_width=f'{cpw_width*1e6}um', prime_gap=f'{cpw_gap*1e6}um',
                                                        second_width=f'{cpw_width*1e6}um', second_gap=f'{cpw_gap*1e6}um',
                                                        open_termination = True))

# q5_read_T = CoupledLineTee(design,'Q5_Read_T', options=dict(pos_x = '5.2mm', pos_y = '2.2mm',
#                                                         orientation = '0',
#                                                         coupling_length = '300um', coupling_space='5um',
#                                                         prime_width=f'{cpw_width*1e6}um', prime_gap=f'{cpw_gap*1e6}um',
#                                                         second_width=f'{cpw_width*1e6}um', second_gap=f'{cpw_gap*1e6}um',
#                                                         open_termination = True))

# q1_read_T = CoupledLineTee(design,'Q1_Read_T', options=dict(pos_x = '1.7mm', pos_y = '2.2mm',
#                                                         orientation = '180',
#                                                         coupling_length = '300um', coupling_space='5um',
#                                                         prime_width=f'{cpw_width*1e6}um', prime_gap=f'{cpw_gap*1e6}um',
#                                                         second_width=f'{cpw_width*1e6}um', second_gap=f'{cpw_gap*1e6}um',
#                                                         open_termination = True))

# q2_read_T = CoupledLineTee(design,'Q2_Read_T', options=dict(pos_x = '5.9mm', pos_y = '2.2mm',
#                                                         orientation = '180',
#                                                         coupling_length = '300um', coupling_space='5um',
#                                                         prime_width=f'{cpw_width*1e6}um', prime_gap=f'{cpw_gap*1e6}um',
#                                                         second_width=f'{cpw_width*1e6}um', second_gap=f'{cpw_gap*1e6}um',
#                                                         open_termination = True))

gui.rebuild()

## Resonators

In [10]:
# connect resnator output coupler to the qubit, this forms a complete readout resonator
cpw_width = 10e-6
cpw_gap = cpw.get_gap_from_width(cpw_width)

read_q4 = RouteMeander(design, 'Read_Q4', options=dict(
    hfss_wire_bonds=True,
    pin_inputs=Dict(
        start_pin=Dict(
            component='Q4',
            pin='readout'
        ),
        end_pin=Dict(
            component='Q4_Read_T',
            pin='second_end'
        )
    ),
    lead=Dict(
        start_straight='150um',
        end_straight='855um'
    ),
    meander=Dict(
        asymmetry='-100um',
        spacing='100um',
    ),
            trace_width=f'{cpw_width*1e6}um',
        trace_gap=f'{cpw_gap*1e6}um',
    fillet="45um",
    total_length='9.583mm'
))


# read_q5 = RouteMeander(design,'Read_Q5', options = dict(hfss_wire_bonds = True,
#                                             pin_inputs=Dict(
#                                                 start_pin=Dict(
#                                                     component='Q5',
#                                                     pin='readout'),
#                                                 end_pin=Dict(
#                                                     component='Q5_Read_T',
#                                                     pin='second_end')
#                                             ),
#                                             lead=Dict(
#                                                 start_straight='125um',
#                                                 end_straight = '125um',
#                                             ),
#                                             meander=Dict(
#                                                 asymmetry = '100um',
#                                                 spacing = "100um",
#                                             ),
#                                              trace_width=f'{cpw_width*1e6}um',
#                                             trace_gap=f'{cpw_gap*1e6}um', 
#                                             fillet = "50um",
#                                             total_length = '7.411mm'))

# read_q1 = RouteMeander(design,'Read_Q1', options = dict(hfss_wire_bonds = True,
#                                             pin_inputs=Dict(
#                                                 start_pin=Dict(
#                                                     component='Q1',
#                                                     pin='readout'),
#                                                 end_pin=Dict(
#                                                     component='Q1_Read_T',
#                                                     pin='second_end')
#                                             ),
#                                             lead=Dict(
#                                                 start_straight='125um',
#                                                 end_straight = '125um',
#                                             ),
#                                             meander=Dict(
#                                                 asymmetry = '0um',
#                                                 spacing = "100um",
#                                             ),
#                                             trace_width=f'{cpw_width*1e6}um', 
#                                             trace_gap=f'{cpw_gap*1e6}um',
#                                             fillet = "50um",
#                                             total_length = '7.531mm'))

# read_q2 = RouteMeander(design,'Read_Q2', options = dict(hfss_wire_bonds = True,
#                                             pin_inputs=Dict(
#                                                 start_pin=Dict(
#                                                     component='Q2',
#                                                     pin='readout'),
#                                                 end_pin=Dict(
#                                                     component='Q2_Read_T',
#                                                     pin='second_end')
#                                             ),
#                                             lead=Dict(
#                                                 start_straight='125um',
#                                                 end_straight = '125um',
#                                             ),
#                                             meander=Dict(
#                                                 asymmetry = '-50um',
#                                                 spacing = "100um",
#                                             ),
#                                             trace_width=f'{cpw_width*1e6}um',
#                                             trace_gap=f'{cpw_gap*1e6}um',
#                                             fillet = "50um",
#                                             total_length = '9.38mm'))



gui.rebuild()
gui.autoscale()

## Feedline completion 

In [11]:
from SQDMetal.Comps.Wires import WirePins, WireElbowParallelPinPin, WirePinStretch 

WirePins(design, 'wirefeed1', Dict(pathObjPins=[('Launch_NW', 'tie'), ('Q4_Read_T', 'prime_start')],
                                                trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))
# WirePins(design, 'wirefeed2', Dict(pathObjPins=[('Q4_Read_T', 'prime_end'), ('Q1_Read_T', 'prime_end')],
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))
# WirePins(design, 'wirefeed3', Dict(pathObjPins=[('Q1_Read_T', 'prime_start'), ('Q5_Read_T', 'prime_start')],
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))
# WirePins(design, 'wirefeed4', Dict(pathObjPins=[('Q5_Read_T', 'prime_end'), ('Q2_Read_T', 'prime_end')],
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))
WirePins(design, 'wirefeed5', Dict(pathObjPins=[('Q4_Read_T', 'prime_end'), ('Launch_NE', 'tie')],
                                                trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um'))
gui.rebuild()
gui.autoscale()

## Charge lines

In [12]:
# from SQDMetal.Comps.Wires import WirePinStretch
# from SQDMetal.Comps.Joints import JointExtend

# chrgln_dist = '400um'
# chrgln_offset = '50um'

# JointExtend(design, 'jntchrg_1', Dict(pos_x = '3.15mm', pos_y = '3.35mm',jointObj='Q1', jointPin='', orientation=90, dist_extend=chrgln_dist, pin_orientation=0))
# JointExtend(design, 'jntchrg_1a', Dict(jointObj='jntchrg_1', jointPin='a', orientation=0, dist_extend=chrgln_offset))
# WirePinStretch(design, 'wirechrg_1', Dict(pin_inputs=Dict(start_pin=Dict(component=f'jntchrg_1a',pin='a')), dist_extend='50um',end_gap='0um', start_gap='0um'));
# WireElbowParallelPinPin(design, 'chrgln1', Dict(pin_inputs=Dict(start_pin=Dict(component=f'Launch_d',pin='tie'), end_pin=Dict(component=f'wirechrg_1',pin='endL')),
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um',frac_pos_elbow='0.3',fillet=0.1))


# JointExtend(design, 'jntchrg_2', Dict(pos_x = '4.15mm', pos_y = '3.45mm',jointObj='Q2', jointPin='', orientation=90, dist_extend=chrgln_dist,pin_orientation=180))
# JointExtend(design, 'jntchrg_2a', Dict(jointObj='jntchrg_2', jointPin='a', orientation=180, dist_extend=chrgln_offset))
# WirePinStretch(design, 'wirechrg_2', Dict(pin_inputs=Dict(start_pin=Dict(component=f'jntchrg_2a',pin='a')), dist_extend='50um',end_gap='0um', start_gap='0um'))
# WireElbowParallelPinPin(design, 'chrgln2', Dict(pin_inputs=Dict(start_pin=Dict(component=f'Launch_c',pin='tie'), end_pin=Dict(component=f'wirechrg_2',pin='endR')),
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um',frac_pos_elbow='0.3',fillet=0.1))


# JointExtend(design, 'jntchrg_5', Dict(pos_x = '3.85mm', pos_y = '0.95mm',jointObj='Q5', jointPin='', orientation=-90, dist_extend=chrgln_dist,pin_orientation=180))
# JointExtend(design, 'jntchrg_5a', Dict(jointObj='jntchrg_5', jointPin='a', orientation=180, dist_extend=chrgln_offset))
# WirePinStretch(design, 'wirechrg_5', Dict(pin_inputs=Dict(start_pin=Dict(component=f'jntchrg_5a',pin='a')), dist_extend='50um',end_gap='0um', start_gap='0um'))
# WireElbowParallelPinPin(design, 'chrgln5', Dict(pin_inputs=Dict(start_pin=Dict(component=f'Launch_b',pin='tie'), end_pin=Dict(component=f'wirechrg_5',pin='endL')),
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um',frac_pos_elbow='0.3',fillet='50um'))



# JointExtend(design, 'jntchrg_4', Dict(pos_x='2.87mm',  pos_y='0.65mm',jointObj='Q4', jointPin='', orientation=-90, dist_extend=chrgln_dist, pin_orientation=0))
# JointExtend(design, 'jntchrg_4a', Dict(jointObj='jntchrg_4', jointPin='a', orientation=0, dist_extend=chrgln_offset))
# WirePinStretch(design, 'wirechrg_4', Dict(pin_inputs=Dict(start_pin=Dict(component=f'jntchrg_4a',pin='a')), dist_extend='50um',end_gap='0um', start_gap='0um'));
# WireElbowParallelPinPin(design, 'chrgln4', Dict(pin_inputs=Dict(start_pin=Dict(component=f'Launch_a',pin='tie'), end_pin=Dict(component=f'wirechrg_4',pin='endR')),
#                                                 trace_width=f'{cpw_width*1e6}um', trace_gap=f'{cpw_gap*1e6}um',frac_pos_elbow='0.3',fillet='50um'));
# gui.rebuild()

## Air-Bridges

In [13]:
# from SQDMetal.Comps.AirBridges import AirBridgeCPW

# AirBridgeCPW(design, 'AB_CL1', Dict(pathObj='chrgln1', layer_bridge=5, dist_air_bridges='310um', dist_air_bridges_ends='50um',pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_CL2', Dict(pathObj='chrgln2', layer_bridge=5, dist_air_bridges='300um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_CL5', Dict(pathObj='chrgln5', layer_bridge=5, dist_air_bridges='300um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_CL4', Dict(pathObj='chrgln4', layer_bridge=5, dist_air_bridges='300um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# AirBridgeCPW(design, 'AB_R1', Dict(pathObj='Read_Q1', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_R2', Dict(pathObj='Read_Q2', layer_bridge=5, dist_air_bridges='510um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_R5', Dict(pathObj='Read_Q5', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_R4', Dict(pathObj='Read_Q4', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# AirBridgeCPW(design, 'AB_FLQ1', Dict(pathObj='Q1_Read_T', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='100um',pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_FLQ2', Dict(pathObj='Q2_Read_T', layer_bridge=5, dist_air_bridges='510um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_FLQ5', Dict(pathObj='Q5_Read_T', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_FLQ4', Dict(pathObj='Q4_Read_T', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# AirBridgeCPW(design, 'AB_FL', Dict(pathObj='wirefeed3', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='100um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# gui.rebuild()
# gui.autoscale()

## Josephson Junction: Dolan Bridge

In [14]:
# from SQDMetal.Comps.Junctions import JunctionDolan

# #Standard Qubit
# JunctionDolan(design, 'JJ_Q1', Dict(pos_x='3.087mm',pos_y='3.003mm', end_x='3.113mm',end_y='3.003mm', bridge_gap='0.2um',  finger_length='1.75um',  finger_width='0.185um',
#                            squid_width='5.4um', stem_width='2um', prong_width='0.5um', prong_length='1.75um', t_pad_size='0.261um',fork_pad_size='0.5um', t_pad_extra='0.um',layer=6))
# #Charge qubit
# JunctionDolan(design, 'JJ_Q2', Dict(pos_x='3.883mm',pos_y='3.05mm', end_x='3.917mm',end_y='3.05mm', bridge_gap='0.2um',  finger_length='1.75um',  finger_width='0.15um',
#                            squid_width='5.4um', stem_width='2um', prong_width='0.5um', prong_length='1.75um', t_pad_size='0.202um',fork_pad_size='0.5um', t_pad_extra='0.um',layer=6))
# #Charge qubit
# JunctionDolan(design, 'JJ_Q4', Dict(pos_x='3.083mm',pos_y='1.3006mm', end_x='3.117mm',end_y='1.3006mm', bridge_gap='0.2um',  finger_length='1.75um',  finger_width='0.15um',
#                            squid_width='5.4um', stem_width='2um', prong_width='0.5um', prong_length='1.75um', t_pad_size='0.202um',fork_pad_size='0.5um', t_pad_extra='0.um',layer=6))
# #Standard Qubit
# JunctionDolan(design, 'JJ_Q5', Dict(pos_x='3.887mm',pos_y='1.3006mm', end_x='3.913mm',end_y='1.3006mm', bridge_gap='0.2um',  finger_length='1.75um',  finger_width='0.185um',
#                            squid_width='5.4um', stem_width='2um', prong_width='0.5um', prong_length='1.75um', t_pad_size='0.261um',fork_pad_size='0.5um', t_pad_extra='0.um',layer=6))
# gui.rebuild()
# gui.autoscale()

## Bandages

In [15]:
# from SQDMetal.Comps.Bandages import BandageRectPin

# BandageRectPin(design, 'B1_Q1', Dict(width='10um', height='5um',target_comp='JJ_Q1', target_pin='t',layer=7))
# BandageRectPin(design, 'B2_Q1', Dict(width='10um', height='5um',target_comp='JJ_Q1', target_pin='f',layer=7))

# BandageRectPin(design, 'B1_Q2', Dict(width='10um', height='5um',target_comp='JJ_Q2', target_pin='t',layer=7))
# BandageRectPin(design, 'B2_Q2', Dict(width='10um', height='5um',target_comp='JJ_Q2', target_pin='f',layer=7))

# BandageRectPin(design, 'B1_Q4', Dict(width='10um', height='5um',target_comp='JJ_Q4', target_pin='t',layer=7))
# BandageRectPin(design, 'B2_Q4', Dict(width='10um', height='5um',target_comp='JJ_Q4', target_pin='f',layer=7))

# BandageRectPin(design, 'B1_Q5', Dict(width='10um', height='5um',target_comp='JJ_Q5', target_pin='t',layer=7))
# BandageRectPin(design, 'B2_Q5', Dict(width='10um', height='5um',target_comp='JJ_Q5', target_pin='f',layer=7))

# gui.rebuild()
# gui.autoscale()

In [16]:
# from SQDMetal.Comps.AirBridges import AirBridgeCPW

# AirBridgeCPW(design, 'AB_CL1', Dict(pathObj='chrgln1', layer_bridge=5, dist_air_bridges='310um', dist_air_bridges_ends='50um',pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_CL2', Dict(pathObj='chrgln2', layer_bridge=5, dist_air_bridges='300um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_CL5', Dict(pathObj='chrgln5', layer_bridge=5, dist_air_bridges='300um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_CL4', Dict(pathObj='chrgln4', layer_bridge=5, dist_air_bridges='300um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# AirBridgeCPW(design, 'AB_R1', Dict(pathObj='Read_Q1', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_R2', Dict(pathObj='Read_Q2', layer_bridge=5, dist_air_bridges='510um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_R5', Dict(pathObj='Read_Q5', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_R4', Dict(pathObj='Read_Q4', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# AirBridgeCPW(design, 'AB_FLQ1', Dict(pathObj='Q1_Read_T', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='100um',pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_FLQ2', Dict(pathObj='Q2_Read_T', layer_bridge=5, dist_air_bridges='510um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_FLQ5', Dict(pathObj='Q5_Read_T', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))
# AirBridgeCPW(design, 'AB_FLQ4', Dict(pathObj='Q4_Read_T', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='50um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

# AirBridgeCPW(design, 'AB_FL', Dict(pathObj='wirefeed3', layer_bridge=5, dist_air_bridges='450um', dist_air_bridges_ends='100um',layers_obj_avoid=[1,2,3],pad_gap='5um',
#                            pad_width='20um',pad_length='40um',bridge_pad_gap='5um',bridge_pad_width='10um',bridge_pad_length='30um',bridge_width='8um'))

gui.rebuild()
gui.autoscale()

# Eigenmode Simulation

In [ ]:
from PALACE_Simulation import PALACE_Simulation

#Eigenmode Simulation Options
eigen_user_options = {
            "mesh_refinement":  0,                                         #refines mesh in PALACE - essetially divides every mesh element in half
            "dielectric_material": "silicon",                              #choose dielectric material - 'silicon' or 'sapphire'
            "starting_freq": 5,                                          #starting frequency in GHz 
            "number_of_freqs": 3,                                          #number of eigenmodes to find
            "solns_to_save": 3,                                            #number of electromagnetic field visualizations to save
            "solver_order": 2,                                             #increasing solver order increases accuracy of simulation, but significantly increases sim time
            "solver_tol": 1.0e-8,                                          #error residual tolerance foriterative solver
            "solver_maxits": 100,                                          #number of solver iterations
            "mesh_max": 170e-3,                                            #maxiumum element size for the mesh in mm
            "mesh_min": 10e-3,                                             #minimum element size for the mesh in mm
            "mesh_sampling": 150,                                          #number of points to mesh along a geometry
            "sim_directory": 'C:\\PALACE_Simulations\\'                    #parent directory where all simulation files will be saved
}

HPC_options = {
            "hpc_nodes": '3',
            "cpus_per_node": '25',
            "sim_time": '3:30:00',
            "sim_memory": '250G',
            "account_name": 'a_fedorov',
            "palace_location": '/scratch/project/palace-sqdlab/Palace-Project-NEW/palace/build_v0.12/bin/palace-x86_64.bin',
            "input_files_location": '/scratch/project/palace-sqdlab/inputFiles/',
            "output_files_location": '/scratch/project/palace-sqdlab/outputFiles/'
}

sim = PALACE_Simulation('Eigenmode', 'eigen_single_qubit', design, eigen_user_options, ports = ['Launch_NW','Launch_NE'], hpc_options = HPC_options)
sim.run_simulation()

Creating geometry for Eigenmode Simulation.
<class 'float'>
Geometry successfully built in Gmsh.
Finely meshing surfaces: [9, 10]
Mesh successfully built in Gmsh.
Directory 'eigen_single_qubit' created at 'C:\PALACE_Simulations\eigen_single_qubit'.
Simulation files created.


In [18]:
import gmsh
gmsh.fltk.run()